# Краулеры

In [148]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
import requests
import pandas as pd

# Журнал №1

In [149]:
session = requests.session()
response = session.get("https://benjamins.com/catalog/sll")
response.status_code

200

In [150]:
user_agent = UserAgent().chrome

url = "https://benjamins.com/catalog/sll"
response = requests.get(url, headers={'User-Agent':user_agent})
articles_block = response.text
soup = BeautifulSoup(articles_block, "html.parser")

hrefs = [item.find('a').get('href') for item in soup.find_all(class_="entry issue")]
print(*hrefs, sep = '\n')

catalog/sll.26.1
catalog/sll.25.2
catalog/sll.25.1
catalog/sll.24.2
catalog/sll.24.1
catalog/sll.23.1-2
catalog/sll.22.2
catalog/sll.22.1
catalog/sll.21.2
catalog/sll.21.1
catalog/sll.20.2
catalog/sll.20.1
catalog/sll.19.2
catalog/sll.19.1
catalog/sll.18.2
catalog/sll.18.1
catalog/sll.17.2
catalog/sll.17.1
catalog/sll.16.2
catalog/sll.16.1
catalog/sll.15.2
catalog/sll.15.1
catalog/sll.14.2
catalog/sll.14.1
catalog/sll.13.2
catalog/sll.13.1
catalog/sll.12.2
catalog/sll.12.1
catalog/sll.11.2
catalog/sll.11.1
catalog/sll.10.2
catalog/sll.10.1
catalog/sll.9.1-2
catalog/sll.8.1-2
catalog/sll.7.2
catalog/sll.7.1
catalog/sll.6.2
catalog/sll.6.1
catalog/sll.5.2
catalog/sll.5.1
catalog/sll.4.1-2
catalog/sll.3.2
catalog/sll.3.1
catalog/sll.2.2
catalog/sll.2.1
catalog/sll.1.2
catalog/sll.1.1


In [151]:
hrefs_ = []

for href in hrefs:
    response = requests.get(f"https://benjamins.com/{href}", headers={'User-Agent':user_agent})
    issue_text = response.text
    soup_issue = BeautifulSoup(issue_text, "html.parser")
    hrefs_ += [item.find('a').get('href') for item in soup_issue.find_all(class_="article") if str(item.find('a').get('href')).startswith("/catalog")]
print(*hrefs_, sep = '\n')

/catalog/sll.21006.sev
/catalog/sll.22002.nov
/catalog/sll.22006.wie
/catalog/sll.00072.mud
/catalog/sll.00074.nav
/catalog/sll.00073.kar
/catalog/sll.00071.yi
/catalog/sll.20012.bem
/catalog/sll.21007.koe
/catalog/sll.21004.gur
/catalog/sll.00068.kar
/catalog/sll.00066.pau
/catalog/sll.00069.vei
/catalog/sll.00070.vil
/catalog/sll.20015.cec
/catalog/sll.20011.loo
/catalog/sll.21001.str
/catalog/sll.00067.gee
/catalog/sll.00063.cat
/catalog/sll.00064.klo
/catalog/sll.00065.mar
/catalog/sll.19002.ebl
/catalog/sll.20006.kar
/catalog/sll.20010.enn
/catalog/sll.20007.rud
/catalog/sll.00061.alm
/catalog/sll.00062.boe
/catalog/sll.00060.cal
/catalog/sll.00059.not
/catalog/sll.19011.wie
/catalog/sll.19014.hud
/catalog/sll.00056.mor
/catalog/sll.00057.for
/catalog/sll.00054.hau
/catalog/sll.00055.kud
/catalog/sll.00058.oom
/catalog/sll.00042.kwo
/catalog/sll.00043.mor
/catalog/sll.00044.pye
/catalog/sll.00045.sch
/catalog/sll.00046.bor
/catalog/sll.00047.dac
/catalog/sll.00048.erg
/catalog/sll

In [152]:
len(hrefs_)

98

In [153]:
hrefs_ += [item.find('a').get('href') for item in soup.find_all(class_="article")]

In [154]:
info = {}

for idx, link in enumerate(hrefs_):
    response = requests.get(f"https://benjamins.com/{link}", headers={'User-Agent':user_agent})
    article_text = response.text
    soup_article = BeautifulSoup(article_text, "html.parser")
    
    article = {}
    article["title"] = soup_article.find(class_=re.compile("(bktitle)|(refsource)")).text
    if soup_article.find(class_="bksubtitle"):
        article["subtitle"] = soup_article.find(class_="bksubtitle").text
    if soup_article.find(class_="bkblurb", itemprop = "description"):
        article["abstract"] = soup_article.find(class_="bkblurb", itemprop = "description").text
    article["author"] = [person.text for person in soup_article.find_all('a', itemprop="author")]
    if soup_article.find_all(class_="bkblurb"):
        article["year"] = re.search(r'[0-9]{4}', soup_article.find_all(class_="bkblurb")[-1].text).group(0)
    if soup_article.find('a', itemprop="sameAs"):
        article["doi"] = soup_article.find('a', itemprop="sameAs").get('href').replace("https://doi.org/", "")
    info[idx] = article
print(len(info))
print(info)

118
{0: {'title': 'Classifiers, argument expression,  and age of acquisition effects in  Turkish Sign Language (TİD)', 'abstract': '\nThis study investigates differences in language production of native, early-learner, and late-learner Turkish Sign Language (Türk İşaret Dili\xa0– TİD) signers in the domain of classifiers. For this study, we conducted a picture-signing task to elicit clauses with classifier constructions from adult Deaf signers of these three groups. The results indicate that there is no significant difference among these three groups with respect to the morphological encoding of thematic roles on verbal roots in classifier constructions. Nonetheless, a difference surfaces in the argument expression patterns among these groups. The data show that the age of exposure to a first linguistic input impacts the argument expression rates as well as which arguments are expressed or left unexpressed. Native signers drop the agent argument more frequently than early-learner and l

In [155]:
d_to_df = {
    'doi': [],
    'author': [],
    'title': [],
    'subtitle': [],
    'year': [],
    'abstract': [],
    'journal': []
}

for k in d_to_df:
    for elem in info.values():
        if k in elem.keys():
            if k == "author":
                elem[k] = ", ".join(elem[k])
            d_to_df[k].append(elem[k])                
        else:
            if k == "journal":
                d_to_df[k].append("SIGN LANGUAGE & LINGUISTICS")
            else:
                d_to_df[k].append(" ")
            
df = pd.DataFrame(data=d_to_df)
df = df.drop_duplicates()
df

,doi,author,title,subtitle,year,abstract,journal
0,10.1075/sll.21006.sev,"Hande Sevgi, Kadir Gökgöz","Classifiers, argument expression, and age of ...",,2023,\nThis study investigates differences in langu...,SIGN LANGUAGE & LINGUISTICS
1,10.1075/sll.22002.nov,"Rama Novogrodsky, Rose Stamp, Sabrin Shaban-Rabah",Word order in simple sentences of tri-lingual...,,2023,\nThis study explores word order patterns prod...,SIGN LANGUAGE & LINGUISTICS
2,10.1075/sll.22006.wie,"Anne Wienholz, Derya Nuhbalaoglu-Ayan, Nivedit...",Neurophysiological evidence for the first men...,,2023,\nAnaphoric pronoun resolution in spoken langu...,SIGN LANGUAGE & LINGUISTICS
3,10.1075/sll.00072.mud,Katie Mudd,How social structure affects the persistence a...,,2023,,SIGN LANGUAGE & LINGUISTICS
4,10.1075/sll.00074.nav,Alexandra Navarrete-González,Focus and contrast in Catalan Sign Language (L...,,2023,,SIGN LANGUAGE & LINGUISTICS
...,...,...,...,...,...,...,...
98,10.1075/sll.22005.bro,Fabian Bross,\nWh-doubling in German Sign Language,Why not sluicing?,2023,\n\nWh-doubling is a widely attested phenomeno...,SIGN LANGUAGE & LINGUISTICS
99,10.1075/sll.00077.gui,Ardavan Guity,Esharani grammatical sketch: An initial descri...,"(Gallaudet University, 2022)",2023,,SIGN LANGUAGE & LINGUISTICS
101,10.1075/sll.00075.koe,Emily A.J. Koenders,The count-mass distinction in Hong Kong Sign ...,A typological study into the surface manifesta...,2023,,SIGN LANGUAGE & LINGUISTICS
102,10.1075/sll.20001.van,"Els van der Kooij, Inge Zwitserlood, Onno Cras...",Strategies for new word formation in NGT,A case for simultaneous morphology,2023,\nHow do new words arise in a sign language? W...,SIGN LANGUAGE & LINGUISTICS


In [156]:
df.to_csv("PR_Sign Language & Linguistics.csv", encoding="utf-8", index=False)

# Журнал №2

In [157]:
session = requests.session()
response = session.get("http://benjamins.com/catalog/lv")
response.status_code

200

In [158]:
user_agent = UserAgent().chrome

url = "https://benjamins.com/catalog/lv"
response = requests.get(url, headers={'User-Agent':user_agent})
articles_block = response.text
soup = BeautifulSoup(articles_block, "html.parser")

hrefs = [item.find('a').get('href') for item in soup.find_all(class_="entry issue")]
print(*hrefs, sep = '\n')

catalog/lv.23.2
catalog/lv.23.1
catalog/lv.22.2
catalog/lv.22.1
catalog/lv.21.2
catalog/lv.21.1
catalog/lv.20.2
catalog/lv.20.1
catalog/lv.19.2
catalog/lv.19.1
catalog/lv.18.2
catalog/lv.18.1
catalog/lv.17.2
catalog/lv.17.1
catalog/lv.16.2
catalog/lv.16.1
catalog/lv.15.2
catalog/lv.15.1
catalog/lv.14.2
catalog/lv.14.1
catalog/lv.13.2
catalog/lv.13.1
catalog/lv.12.2
catalog/lv.12.1
catalog/lv.11.2
catalog/lv.11.1


In [159]:
hrefs_ = []

for href in hrefs:
    response = requests.get(f"https://benjamins.com/{href}", headers={'User-Agent':user_agent})
    issue_text = response.text
    soup_issue = BeautifulSoup(issue_text, "html.parser")
    hrefs_ += [item.find('a').get('href') for item in soup_issue.find_all(class_="article") if str(item.find('a').get('href')).startswith("/catalog")]
print(*hrefs_, sep = '\n')

/catalog/lv.20015.mun
/catalog/lv.22008.and
/catalog/lv.23037.ma
/catalog/lv.21004.ter
/catalog/lv.20019.kal
/catalog/lv.21003.gru
/catalog/lv.20016.met
/catalog/lv.21001.iri
/catalog/lv.20013.lys
/catalog/lv.20007.fuc
/catalog/lv.20009.iri
/catalog/lv.19007.cal
/catalog/lv.20002.toq
/catalog/lv.19012.qua
/catalog/lv.00039.int
/catalog/lv.00032.sve
/catalog/lv.00033.pre
/catalog/lv.00034.rea
/catalog/lv.00035.fra
/catalog/lv.00036.mat
/catalog/lv.00037.gib
/catalog/lv.00038.ozt
/catalog/lv.00031.ded
/catalog/lv.00015.lim
/catalog/lv.00016.sto
/catalog/lv.00017.god
/catalog/lv.00018.san
/catalog/lv.00019.fra
/catalog/lv.00020.nev
/catalog/lv.00021.tho
/catalog/lv.00022.sal
/catalog/lv.00023.bar
/catalog/lv.00024.bom
/catalog/lv.00025.gal
/catalog/lv.00026.cou
/catalog/lv.00027.cha
/catalog/lv.00028.san
/catalog/lv.00029.gue
/catalog/lv.00030.cou
/catalog/lv.00013.edi
/catalog/lv.16011.sza
/catalog/lv.16013.kuk
/catalog/lv.16010.bar
/catalog/lv.16012.bac
/catalog/lv.17004.ber
/catalog/lv

In [160]:
len(hrefs_)

85

In [161]:
hrefs_ += [item.find('a').get('href') for item in soup.find_all(class_="article")]

In [162]:
info = {}

for idx, link in enumerate(hrefs_):
    response = requests.get(f"https://benjamins.com/{link}", headers={'User-Agent':user_agent})
    article_text = response.text
    soup_article = BeautifulSoup(article_text, "html.parser")
    
    article = {}
    article["title"] = soup_article.find(class_=re.compile("(bktitle)|(refsource)")).text
    if soup_article.find(class_="bksubtitle"):
        article["subtitle"] = soup_article.find(class_="bksubtitle").text
    if soup_article.find(class_="bkblurb", itemprop = "description"):
        article["abstract"] = soup_article.find(class_="bkblurb", itemprop = "description").text
    article["author"] = [person.text for person in soup_article.find_all('a', itemprop="author")]
    if soup_article.find_all(class_="bkblurb"):
        article["year"] = re.search(r'[0-9]{4}', soup_article.find_all(class_="bkblurb")[-1].text).group(0)
    if soup_article.find('a', itemprop="sameAs"):
        article["doi"] = soup_article.find('a', itemprop="sameAs").get('href').replace("https://doi.org/", "")
    info[idx] = article
print(len(info))
print(info)

92
{0: {'title': 'Towards a syntactic understanding  of connective particles', 'subtitle': 'The final pero phenomenon in Bahiense Spanish', 'abstract': '\nThe contrastive connector pero ‘but’ is rigidly sentence-initial in most Spanish varieties. However, at least three Spanish dialects allow locating it at the end of a sentence. This paper discusses the properties of final pero as attested in the dialect spoken in Bahia Blanca (Argentina), i.e., the so-called pero bahiense. First, I demonstrate that pero bahiense cannot be reduced to superficially similar phenomena in Spanish. Then, I offer a comparison between pero bahiense and its sentence-initial counterpart showing that they share a number of non-trivial characteristics but also differ in relevant regards. Based on these properties, I advance an account of the pero bahiense phenomenon according to which instances of pero that express concessivity may optionally attract the CP projection to their left. While the analysis does not c

In [163]:
d_to_df = {
    'doi': [],
    'author': [],
    'title': [],
    'subtitle': [],
    'year': [],
    'abstract': [],
    'journal': []
}

for k in d_to_df:
    for elem in info.values():
        if k in elem.keys():
            if k == "author":
                elem[k] = ", ".join(elem[k])
            d_to_df[k].append(elem[k])                
        else:
            if k == "journal":
                d_to_df[k].append("LINGUISTIC VARIATION")
            else:
                d_to_df[k].append(" ")
            
df = pd.DataFrame(data=d_to_df)
df = df.drop_duplicates()
df

,doi,author,title,subtitle,year,abstract,journal
0,10.1075/lv.20015.mun,Carlos Muñoz Pérez,Towards a syntactic understanding of connecti...,The final pero phenomenon in Bahiense Spanish,2022,\nThe contrastive connector pero ‘but’ is rigi...,LINGUISTIC VARIATION
1,10.1075/lv.22008.and,"Alexander Andrason, Admire Phiri",Talking to animals in a moribund language,"Pragma-semantics, phonetics, and morphology o...",2022,\nThe present article is dedicated to conative...,LINGUISTIC VARIATION
2,10.1075/lv.23037.ma,Yanxiao Ma,The Oxford Handbook of Universal Grammar,,2023,,LINGUISTIC VARIATION
3,10.1075/lv.21004.ter,"Alexandra Ter-Avanesova, Michael Daniel",The second genitive in the history of Russian...,,2022,"\nIn this paper, we provide a survey of the di...",LINGUISTIC VARIATION
4,10.1075/lv.20019.kal,"Andra Kalnača, Ilze Lokmane",Partitive genitive constructions and agreemen...,,2022,\nThe article takes a closer look at the parti...,LINGUISTIC VARIATION
...,...,...,...,...,...,...,...
86,10.1075/lv.22001.lip,"Anikó Lipták, Rint Sybesma",Missing and not found,What adjectival agreement reveals about determ...,2023,\nThis paper offers novel insights on articlel...,LINGUISTIC VARIATION
87,10.1075/lv.22006.esp,"M. Teresa Espinal, Cristina Real-Puigdollers, ...",From a movement verb to an epistemic discourse...,The diachronic change of Spanish vaya\n,2022,\nBesides its main use as a form of the moveme...,LINGUISTIC VARIATION
88,10.1075/lv.21014.ver,"Philip C. Vergeiner, Konstantin Niehaus",Article use in Upper German – a ‘radical’ stag...,New findings from Austria,2022,\nDespite an increasing interest in German dia...,LINGUISTIC VARIATION
89,10.1075/lv.22005.bro,Hans Broekhuis,VO or OV,V to v or not to v\n,2022,\nThis article sketches a new analysis of the ...,LINGUISTIC VARIATION


In [164]:
df.to_csv("PR_Linguistic Variation.csv", encoding="utf-8", index=False)

# Журнал №3

In [165]:
session = requests.session()
response = session.get("https://www.ling.sinica.edu.tw/item/en?code=list&ids=4")
response.status_code

200

In [166]:
info = {}
user_agent = UserAgent().chrome
idx = 0

for i in range(1, 25):
    for j in range(1, 5):
        url = f"https://www.ling.sinica.edu.tw/item/en?act=journal&code=directory&volume={i}&period={j}"
        response = requests.get(url, headers={'User-Agent':user_agent})
        articles_block = response.text
        soup_block = BeautifulSoup(articles_block, "html.parser")
        soup_article = soup_block.find_all(class_="artcont")
        date = soup_block.find('ul', {'class': "speech_con"}).find_all('li')[1].text.replace("Publication Date", "")
        for soup in soup_article:
            article = {}
            article["title"] = soup.find_all("p")[0].text
            article["author"] = soup.find_all("p")[1].text
            article["year"] = date[:4]
            info[idx] = article
            idx += 1
            
print(len(info))
print(info)

655
{0: {'title': 'Segregatories and reciprocals and their scopes: Interfacing constructions in short-term memory with long-term memory through their semantic features', 'author': 'Robert L. Cheng', 'year': '2000'}, 1: {'title': 'Morphologization of the degree adverb HEN', 'author': 'Kawai Chui', 'year': '2000'}, 2: {'title': 'The subject chain in a compositional cognitive grammar', 'author': 'Hsin-I Hsieh', 'year': '2000'}, 3: {'title': 'A multi-dimensional analysis of spoken and written Taiwanese register ', 'author': 'Hak-khiam Tiun', 'year': '2000'}, 4: {'title': 'A frame-based account of lexical polysemy in Taiwanese', 'author': 'Chinfa Lien', 'year': '2000'}, 5: {'title': 'Agreement and disagreement of vowel features: Mid vowel assimilation in Yanggu', 'author': 'Yen-Hwei Lin', 'year': '2000'}, 6: {'title': 'Parallelisms in the verb morphology of Sidaba rGyalrong and Lavrung in rGyalrongic', 'author': 'Jackson T.-S. Sun', 'year': '2000'}, 7: {'title': 'Finite and nonfinite clause

In [167]:
d_to_df = {
    'author': [],
    'title': [],
    'year': [],
    'journal': []
}

for k in d_to_df:
    for elem in info.values():
        if k in elem.keys():
            d_to_df[k].append(elem[k])
        elif k == 'journal':
            d_to_df[k].append("LANGUAGE AND LINGUISTICS")
        else:
            d_to_df[k].append(" ")
            
df = pd.DataFrame(data=d_to_df)
df = df.drop_duplicates()
df

,author,title,year,journal
0,Robert L. Cheng,Segregatories and reciprocals and their scopes...,2000,LANGUAGE AND LINGUISTICS
1,Kawai Chui,Morphologization of the degree adverb HEN,2000,LANGUAGE AND LINGUISTICS
2,Hsin-I Hsieh,The subject chain in a compositional cognitive...,2000,LANGUAGE AND LINGUISTICS
3,Hak-khiam Tiun,A multi-dimensional analysis of spoken and wri...,2000,LANGUAGE AND LINGUISTICS
4,Chinfa Lien,A frame-based account of lexical polysemy in T...,2000,LANGUAGE AND LINGUISTICS
...,...,...,...,...
650,Mathieu Beaudouin,Tangut and Horpa languages: Some shared morpho...,2023,LANGUAGE AND LINGUISTICS
651,Cherry Chit-yu Lam,Rethinking postverbal 'acquire' and related co...,2023,LANGUAGE AND LINGUISTICS
652,Meili Liu & Hubert Cuyckens,The grammaticalization of impossibility: The e...,2023,LANGUAGE AND LINGUISTICS
653,Jianqiang Sun,The {bei+verb+jiangqu} construction in pre-twe...,2023,LANGUAGE AND LINGUISTICS


In [168]:
df.to_csv("PR_Language and Linguistics.csv", encoding="utf-8", index=False)

# Журнал №4

In [169]:
session = requests.session()
response = session.get("https://suvlin.ffzg.hr/index.php/hr/")
response.status_code

200

In [170]:
user_agent = UserAgent().chrome

url = "https://suvlin.ffzg.hr/index.php/hr/arhiva-casopisa/mapa-web-stranica"
response = requests.get(url, headers={'User-Agent':user_agent})
articles_block = response.text
soup = BeautifulSoup(articles_block, "html.parser")

hrefs = []

uls = soup.find_all('ul')
for ul in uls:
    for li in ul.find_all("li"):
        if (li.find(class_="level_1") or li.find(class_="level_2")) and li.find("ul"):
            sec_uls = li.find_all("ul")
            for sec_ul in sec_uls:
                for sec_li in sec_ul.find_all("li"):
                    hrefs.append(sec_li.find('a').get('href'))
hrefs = list(set(hrefs))
print(len(hrefs))

459


In [171]:
info = {}

for idx, link in enumerate(hrefs):
    response = requests.get(f"https://suvlin.ffzg.hr{link}", headers={'User-Agent':user_agent})
    article_text = response.text
    soup_article = BeautifulSoup(article_text, "html.parser")
    
    article = {}
    if soup_article.find(class_="page-header"):
        article["title"] = soup_article.find(class_="page-header").text
    article["abstract"] = soup_article.find_all('p')[5].text
    article["author"] = [person.text for person in soup_article.find_all('strong') if "Adresa" not in person.text and "Glavni" not in person.text]
    if soup_article.find(class_="phocadownloadfile32"):
        download = soup_article.find(class_="phocadownloadfile32").find('a').text
        if re.search(r"\(.+\)", download):
            year = re.search(r"\(.+\)", download).group(0).replace("(", "").replace(")", "")
            if year[-4:].isdigit():
                article["year"] = year[-4:]
    if soup_article.find('a', {'title' : 'doi'}):
        article["doi"] = str(soup_article.find('a', {'title' : 'doi'}).get("href")).replace("https://doi.org/", "")

    info[idx] = article
print(len(info))
print(info)

459
{0: {'title': '\n\t\t\t\t\tVol. 49-50  No 1. (2000.)\n\n\t\t\t\t\t\t\tBroj članka:\xa0\n\t\t\t\t\t\t\t14\t\t\t\t\t\t\n', 'abstract': '\r\n                All                369921\n', 'author': []}, 1: {'title': '\n\n\t\t\t\tOd neurona do metafore (i natrag): Neuralna teorija metafore u okviru neuralne teorije jezika i mišljenja\t\t\t\n', 'abstract': "U radu se prvo daje kratak pregled istraživanja i pravaca koji su utjecali na oblikovanjeneuralne teorije jezika i mišljenja: kognitivizam nasuprot objektivizmu, funkcionalizam('jaka' umjetna inteligencija nasuprot utjelovljenoj umjetnoj inteligenciji), kategorizacija,utjelovljena kognicija i otkriće zrcalnih neurona. Zatim se daje pregled temeljnih postavkii pojmova neuralne teorije jezika i mišljenja: odnos neuralne teorije jezika i kognitivnelingvistike, simulacijska semantika J. Feldmana, tipovi neuronskih mreža, tipovi neuralnogučenja, utjelovljena konstrukcijska gramatika. Na kraju se pozornost usmjerava metaforii metaforičkomu 

In [172]:
d_to_df = {
    'doi': [],
    'author': [],
    'title': [],
    'year': [],
    'abstract': [],
    'journal': []
}

for k in d_to_df:
    for elem in info.values():
        if k in elem.keys():
            if k == "author":
                elem[k] = ", ".join(elem[k])
            d_to_df[k].append(elem[k])                
        else:
            if k == "journal":
                d_to_df[k].append("SUVREMENA LINGVISTIKA")
            else:
                d_to_df[k].append(" ")
            
df = pd.DataFrame(data=d_to_df)
df = df.drop_duplicates()
df

,doi,author,title,year,abstract,journal
0,,,\n\t\t\t\t\tVol. 49-50 No 1. (2000.)\n\n\t\t\...,,\r\n All 369921\n,SUVREMENA LINGVISTIKA
1,,Kristina Štrkalj Despot,\n\n\t\t\t\tOd neurona do metafore (i natrag):...,,U radu se prvo daje kratak pregled istraživanj...,SUVREMENA LINGVISTIKA
2,,Shahzaman Haque,\n\n\t\t\t\tRazlike u jezičnoj politici između...,,,SUVREMENA LINGVISTIKA
3,,"Marina Grubišić, Gitte Kristiansen i René Dirv...",\n\n\t\t\t\tGitte Kristiansen i René Dirven (u...,,Prikaz,SUVREMENA LINGVISTIKA
4,,"Florin-Mihai Dat, Arijana Krišković, Taoufik M...",\n\n\n\t\t\t\t\t\tMetateza i homonimija u bibl...,,\n\n \n\t\t\tOpširnije:\t\t\tSemantički prijen...,SUVREMENA LINGVISTIKA
...,...,...,...,...,...,...
454,,Robert D. Van Valin,\n\n\t\t\t\tAnaliza tromjesnih predikata na te...,,,SUVREMENA LINGVISTIKA
455,,Sunil Kumar Bhatt,\n\n\t\t\t\tThe concept of Completivity (Perfe...,,,SUVREMENA LINGVISTIKA
456,,Krešimir Mićanović,\n\n\t\t\t\tPosvojni pridjevi i izražavanje po...,,,SUVREMENA LINGVISTIKA
457,,"Gordana Varošanec–Škarić, Iva Pavić, Gabrijela...",\n\n\t\t\t\tIndeksi sličnosti i različitosti k...,,\r\n Today 24\n,SUVREMENA LINGVISTIKA


In [173]:
df.to_csv("PR_Suvremena Lingvistika.csv", encoding="utf-8", index=False)

# Журнал №5

In [174]:
session = requests.session()
response = session.get("https://benjamins.com/catalog/jhl")
response.status_code

200

In [175]:
user_agent = UserAgent().chrome

url = "https://benjamins.com/catalog/jhl"
response = requests.get(url, headers={'User-Agent':user_agent})
articles_block = response.text
soup = BeautifulSoup(articles_block, "html.parser")

hrefs = [item.find('a').get('href') for item in soup.find_all(class_="entry issue")]
print(*hrefs, sep = '\n')

catalog/jhl.13.3
catalog/jhl.13.2
catalog/jhl.13.1
catalog/jhl.12.3
catalog/jhl.12.2
catalog/jhl.12.1
catalog/jhl.11.3
catalog/jhl.11.2
catalog/jhl.11.1
catalog/jhl.10.3
catalog/jhl.10.2
catalog/jhl.10.1
catalog/jhl.9.3
catalog/jhl.9.2
catalog/jhl.9.1
catalog/jhl.8.3
catalog/jhl.8.2
catalog/jhl.8.1
catalog/jhl.7.3
catalog/jhl.7.1-2
catalog/jhl.6.2
catalog/jhl.6.1
catalog/jhl.5.2
catalog/jhl.5.1
catalog/jhl.4.2
catalog/jhl.4.1
catalog/jhl.3.2
catalog/jhl.3.1
catalog/jhl.2.2
catalog/jhl.2.1
catalog/jhl.1.2
catalog/jhl.1.1


In [176]:
hrefs_ = []

for href in hrefs:
    response = requests.get(f"https://benjamins.com/{href}", headers={'User-Agent':user_agent})
    issue_text = response.text
    soup_issue = BeautifulSoup(issue_text, "html.parser")
    hrefs_ += [item.find('a').get('href') for item in soup_issue.find_all(class_="article") if str(item.find('a').get('href')).startswith("/catalog")]
print(*hrefs_, sep = '\n')

/catalog/jhl.21033.san
/catalog/jhl.22014.lyu
/catalog/jhl.22026.mos
/catalog/jhl.22033.col
/catalog/jhl.22017.rak
/catalog/jhl.21003.wan
/catalog/jhl.22001.koc
/catalog/jhl.21046.law
/catalog/jhl.21028.ama
/catalog/jhl.21026.mei
/catalog/jhl.21025.lee
/catalog/jhl.21008.bea
/catalog/jhl.00013.edi
/catalog/jhl.20044.ang
/catalog/jhl.20054.bos
/catalog/jhl.21019.kha
/catalog/jhl.21020.chi
/catalog/jhl.22016.alv
/catalog/jhl.21007.gia
/catalog/jhl.20027.blu
/catalog/jhl.20031.maz
/catalog/jhl.20061.kra
/catalog/jhl.20023.yan
/catalog/jhl.20045.ung
/catalog/jhl.20058.sme
/catalog/jhl.20024.ver
/catalog/jhl.20011.str
/catalog/jhl.20020.led
/catalog/jhl.20002.sei
/catalog/jhl.19041.man
/catalog/jhl.19010.bir
/catalog/jhl.19028.ack
/catalog/jhl.19011.opp
/catalog/jhl.21012.dah
/catalog/jhl.20016.ald
/catalog/jhl.19046.noo
/catalog/jhl.20017.jan
/catalog/jhl.21005.mat
/catalog/jhl.20018.fed
/catalog/jhl.20028.goe
/catalog/jhl.19011.opp
/catalog/jhl.19021.bod
/catalog/jhl.19024.dja
/catalog/jh

In [177]:
len(hrefs_)

117

In [178]:
hrefs_ += [item.find('a').get('href') for item in soup.find_all(class_="article")]

In [179]:
info = {}

for idx, link in enumerate(hrefs_):
    response = requests.get(f"https://benjamins.com/{link}", headers={'User-Agent':user_agent})
    article_text = response.text
    soup_article = BeautifulSoup(article_text, "html.parser")
    
    article = {}
    article["title"] = soup_article.find(class_=re.compile("(bktitle)|(refsource)")).text
    if soup_article.find(class_="bksubtitle"):
        article["subtitle"] = soup_article.find(class_="bksubtitle").text
    if soup_article.find(class_="bkblurb", itemprop = "description"):
        article["abstract"] = soup_article.find(class_="bkblurb", itemprop = "description").text
    article["author"] = [person.text for person in soup_article.find_all('a', itemprop="author")]
    if soup_article.find_all(class_="bkblurb"):
        article["year"] = re.search(r'[0-9]{4}', soup_article.find_all(class_="bkblurb")[-1].text).group(0)
    if soup_article.find('a', itemprop="sameAs"):
        article["doi"] = soup_article.find('a', itemprop="sameAs").get('href').replace("https://doi.org/", "")
    info[idx] = article
print(len(info))
print(info)

151
{0: {'title': 'Children as agents of language change', 'subtitle': 'Diachronic evidence from Latin American Spanish phonology', 'abstract': '\nThis paper explores the operation of child language acquisition as a critical factor in some forms of language change. It proposes a sociohistorical model that incorporates the potential for young children to function as linguistic agents in certain environments, characterized by unpredictable variation in the input, lack of normative mechanisms, and the possibility for the emergence of peer networks among children. The model is then applied to explain a well-documented but poorly understood phonological change in the history of Latin American Spanish: the simplification of the system of sibilants in 16th-century Colonial Spanish. This change was nestled in ecological environments characterized by intense contact among L1 and L2 speakers of several varieties of Iberian and non-Iberian languages, as well as the rapid breakdown and reshaping o

In [180]:
d_to_df = {
    'doi': [],
    'author': [],
    'title': [],
    'year': [],
    'abstract': [],
    'journal': []
}

for k in d_to_df:
    for elem in info.values():
        if k in elem.keys():
            if k == "author":
                elem[k] = ", ".join(elem[k])
            d_to_df[k].append(elem[k])                
        else:
            if k == "journal":
                d_to_df[k].append("JOURNAL OF HISTORICAL LINGUISTICS")
            else:
                d_to_df[k].append(" ")
            
df = pd.DataFrame(data=d_to_df)
df = df.drop_duplicates()
df

,doi,author,title,year,abstract,journal
0,10.1075/jhl.21033.san,"Israel Sanz-Sánchez, María Irene Moyna",Children as agents of language change,2022,\nThis paper explores the operation of child l...,JOURNAL OF HISTORICAL LINGUISTICS
1,10.1075/jhl.22014.lyu,"Ekaterina Lyutikova, Andrei Sideltsev",Relative construction in Hittite,2022,\nThe paper proposes a novel structural analys...,JOURNAL OF HISTORICAL LINGUISTICS
2,10.1075/jhl.22026.mos,Ann-Marie Moser,The ups and downs of relative particles in Ge...,2023,\nThe aim and scope of this article is to take...,JOURNAL OF HISTORICAL LINGUISTICS
3,10.1075/jhl.22033.col,Vincent Collette,‘Common nighthawk’ (Chordeiles minor) in Algo...,2023,\nSome North American indigenous languages hav...,JOURNAL OF HISTORICAL LINGUISTICS
4,10.1075/jhl.22017.rak,György Rákosi,The NP-strategy for Expressing Reciprocity: Ty...,2023,,JOURNAL OF HISTORICAL LINGUISTICS
...,...,...,...,...,...,...
127,10.1075/jhl.22031.sch,Stefan Schneider,Construct types in language change,2023,\nThis article combines ideas and concepts der...,JOURNAL OF HISTORICAL LINGUISTICS
128,10.1075/jhl.22048.lar,Ezra la Roi,Polarity reversal constructions and counterfac...,2023,\nPolarity reversal has recently been argued t...,JOURNAL OF HISTORICAL LINGUISTICS
131,10.1075/jhl.21056.ras,"Mohammad Rasekh-Mahand, Mehdi Parizadeh",Different functions of ‘rā’ in New Persian,2023,"\nPersian has a polyfunctional case marker, ‘r...",JOURNAL OF HISTORICAL LINGUISTICS
132,10.1075/jhl.22032.yam,Aaron Yamada,A diachronic analysis of Spanish alg- series a...,2023,\nWhile previous studies have analyzed the cha...,JOURNAL OF HISTORICAL LINGUISTICS


In [181]:
df.to_csv("PR_Journal of Historical Linguistics.csv", encoding="utf-8", index=False)